# LAB: Workflow de Datos. Limpieza y sumarización de datos para el Desafío 3.

## Introducción

La idea de este lab es comenzar a trabajar con los datos del Desafío 3. Vamos a tratar de comenzar con el proceso de limpieza, sumarización y análisis exploratorio del dataset de Properatti. Como recordarán, el objetivo final es el desarrollo de un tasador automático a ser aplicado a las próximas propiedades que sean comercializadas por la empresa mediante un modelo de regresión. Como verán, el dataset está sumamente "sucio" y require un intenso proceso de limpieza.

El dataset contiene información referida al primer semestre de 2017 de las propiedades comercializables por la empresa:

* Fecha de creación
* Tipo de la propiedad (house, apartment, ph)
* Operación del aviso (sell, rent)
* Nombre del lugar
* Nombre del lugar + nombre de sus ‘padres’
* ID de geonames del lugar (si está disponible)
* Latitud,Longitud
* Precio original del aviso
* Moneda original del aviso (ARS, USD)
* Precio del aviso en moneda local (ARS)
* Precio aproximado en USD
* Superficie en m²
* Superficie cubierta en m²
* Precio en USD/m²
* Precio por m²
* N° de piso, si corresponde
* Ambientes
* URL en Properati
* Descripción
* Título
* URL de un thumbnail de la primer foto


Una vez terminada la limpieza, aquellos que se animen pueden empezar a probar algunos modelos simples de regresión.


## Objetivos de aprendizaje

* Practicar técnicas de limpieza de datos de tipo texto
* Practicar conversiones de tipo de dato
* Practicar rellenar datos faltantes con 0s o promedios
* Practicar técnicas para trabajo con datos categóricos
* Transformar datos a información útil
* Realizar un primer análisis exploratorio
* BONUS: realizar un primer modelo de regresión

In [1]:
% matplotlib inline
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
import re

In [2]:
from ml_utils import plots

In [4]:
# Cargar la data
df = pd.read_csv('../data/properatti.csv')  # DATOS DE PROPIEDADES EN VENTA PROPERATI
print(df.columns)
df.head()

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [5]:
b_norte_df = df[(df["state_name"] == "Capital Federal") & (df["property_type"]=="apartment") & (df["place_name"] == "Barrio Norte") | ((df["place_name"] == "Recoleta") | (df["place_name"] == "Retiro"))]
b_norte_df.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
178,178,sell,apartment,Barrio Norte,|Argentina|Capital Federal|Barrio Norte|,Argentina,Capital Federal,3436109.0,"-34.594304,-58.403685",-34.594304,...,35.0,NaN,2657.142857,NaN,1.0,NaN,http://www.properati.com.ar/15dd7_venta_depart...,Corredor Responsable: Alejandro Botta - CUCICB...,"Venta Mono dividido, amoblado, con muy buena r...",https://thumbs4.properati.com/8/mpj88w7IJuYqlw...
200,200,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5994274,-58.4015856",-34.599427,...,196.0,1938.775510,1938.775510,NaN,7.0,NaN,http://www.properati.com.ar/15dhm_venta_depart...,Los equipos de aire acondicionado no estan inc...,Semipiso. Living. Comedor. 4 dormitorio(s) 1 e...,https://thumbs4.properati.com/3/Iokg3DyiJu3v_e...
227,227,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5883487899,-58.3900529339",-34.588349,...,181.0,3246.073298,3425.414365,NaN,5.0,NaN,http://www.properati.com.ar/15djy_venta_depart...,"Espectacular ubicación, al frente orientación ...",Recoleta Quintana Ancha 5 Amb,https://thumbs4.properati.com/2/AZGb5-RYl59wVS...
255,255,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5955984,-58.4127129",-34.595598,...,33.0,3636.363636,3636.363636,NaN,1.0,NaN,http://www.properati.com.ar/15dpd_venta_depart...,Excelente departamento a estrenar en recoleta ...,Sanchez de Bustamante 1300-Recoleta,https://thumbs4.properati.com/0/FkdP8ymn9D3IuB...
262,262,sell,apartment,Barrio Norte,|Argentina|Capital Federal|Barrio Norte|,Argentina,Capital Federal,3436109.0,"-34.5837821,-58.4042275",-34.583782,...,37.0,2648.648649,2648.648649,NaN,3.0,NaN,http://www.properati.com.ar/15dqa_venta_depart...,Departamento. Living/Comedor. 2 dormitorio(s)....,Departamento. Living/Comedor. 2 dormitorio(s)....,https://thumbs4.properati.com/3/uPErfXjjHg3fyN...


In [5]:
b_norte_df = df[(df['state_name'] == 'Capital Federal') & (df['property_type'] == 'apartment') & ((df['place_name'] == 'Retiro') | (df['place_name'] == 'Recoleta') )]
b_norte_df.columns = ['indice', 'operacion', 'tipo_propiedad', 'barrio',
       'resumen_zona', 'pais', 'provincia', 'geonames_id',
       'latitud_longitud', 'latitud', 'longitud', 'precio', 'moneda',
       'cambio_aproximado', 'precio_usd', 'superficie_m2',
       'superficie_cubierta', 'precio_usd_m2', 'precio_m2', 'piso',
       'habitaciones', 'expensas', 'properati_url', 'descripcion', 'titulo',
       'imagen']
b_norte_df.head()

,indice,operacion,tipo_propiedad,barrio,resumen_zona,pais,provincia,geonames_id,latitud_longitud,latitud,...,superficie_cubierta,precio_usd_m2,precio_m2,piso,habitaciones,expensas,properati_url,descripcion,titulo,imagen
200,200,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5994274,-58.4015856",-34.599427,...,196.0,1938.775510,1938.775510,NaN,7.0,NaN,http://www.properati.com.ar/15dhm_venta_depart...,Los equipos de aire acondicionado no estan inc...,Semipiso. Living. Comedor. 4 dormitorio(s) 1 e...,https://thumbs4.properati.com/3/Iokg3DyiJu3v_e...
227,227,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5883487899,-58.3900529339",-34.588349,...,181.0,3246.073298,3425.414365,NaN,5.0,NaN,http://www.properati.com.ar/15djy_venta_depart...,"Espectacular ubicación, al frente orientación ...",Recoleta Quintana Ancha 5 Amb,https://thumbs4.properati.com/2/AZGb5-RYl59wVS...
255,255,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5955984,-58.4127129",-34.595598,...,33.0,3636.363636,3636.363636,NaN,1.0,NaN,http://www.properati.com.ar/15dpd_venta_depart...,Excelente departamento a estrenar en recoleta ...,Sanchez de Bustamante 1300-Recoleta,https://thumbs4.properati.com/0/FkdP8ymn9D3IuB...
412,412,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.590333,-58.3864771",-34.590333,...,235.0,2936.170213,2936.170213,NaN,5.0,NaN,http://www.properati.com.ar/15ehd_venta_depart...,2º piso al frente con balcón y grandes ventana...,Montevideo 1700 entre Alvear y Quintana,https://thumbs4.properati.com/7/mzfxgkFbcL6syg...
422,422,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5910354,-58.4006232",-34.591035,...,136.0,2700.000000,2779.411765,NaN,4.0,NaN,http://www.properati.com.ar/15eig_venta_depart...,Edificio de categoria en excelentes condicione...,Departamento venta,https://thumbs4.properati.com/2/-FUozvmum1Wy4n...


geonames_id pagina: http://www.geonames.org/3430787/warnham-flats.html

In [6]:
b_norte_df.shape

(1660, 26)

In [7]:
b_norte_filtrado = b_norte_df[['tipo_propiedad', 'barrio',
                               'latitud', 'longitud', 'precio', 'moneda',
       'cambio_aproximado', 'precio_usd', 'superficie_m2',
       'superficie_cubierta', 'precio_usd_m2', 'precio_m2', 'piso',
       'habitaciones', 'expensas', 'properati_url', 'descripcion', 'titulo']]
b_norte_filtrado.head()

,tipo_propiedad,barrio,latitud,longitud,precio,moneda,cambio_aproximado,precio_usd,superficie_m2,superficie_cubierta,precio_usd_m2,precio_m2,piso,habitaciones,expensas,properati_url,descripcion,titulo
200,apartment,Recoleta,-34.599427,-58.401586,380000.0,USD,6704910.0,380000.0,196.0,196.0,1938.775510,1938.775510,NaN,7.0,NaN,http://www.properati.com.ar/15dhm_venta_depart...,Los equipos de aire acondicionado no estan inc...,Semipiso. Living. Comedor. 4 dormitorio(s) 1 e...
227,apartment,Recoleta,-34.588349,-58.390053,620000.0,USD,10939590.0,620000.0,191.0,181.0,3246.073298,3425.414365,NaN,5.0,NaN,http://www.properati.com.ar/15djy_venta_depart...,"Espectacular ubicación, al frente orientación ...",Recoleta Quintana Ancha 5 Amb
255,apartment,Recoleta,-34.595598,-58.412713,120000.0,USD,2117340.0,120000.0,33.0,33.0,3636.363636,3636.363636,NaN,1.0,NaN,http://www.properati.com.ar/15dpd_venta_depart...,Excelente departamento a estrenar en recoleta ...,Sanchez de Bustamante 1300-Recoleta
412,apartment,Recoleta,-34.590333,-58.386477,690000.0,USD,12174705.0,690000.0,235.0,235.0,2936.170213,2936.170213,NaN,5.0,NaN,http://www.properati.com.ar/15ehd_venta_depart...,2º piso al frente con balcón y grandes ventana...,Montevideo 1700 entre Alvear y Quintana
422,apartment,Recoleta,-34.591035,-58.400623,378000.0,USD,6669621.0,378000.0,140.0,136.0,2700.000000,2779.411765,NaN,4.0,NaN,http://www.properati.com.ar/15eig_venta_depart...,Edificio de categoria en excelentes condicione...,Departamento venta


In [8]:
b_norte_filtrado.shape

(1660, 18)

In [9]:
print(b_norte_filtrado["expensas"].dtype)
aux = (b_norte_filtrado["expensas"].iloc[0])

float64


In [10]:
print(aux)

nan


# Palabras abstractas positivas

In [74]:
positivo = ["luminoso", "espacioso", "buen estado", "amenities", "ammenities"]


In [75]:
b_norte_expresion = b_norte_filtrado.copy()
b_norte_anios = b_norte_expresion.copy()
for palabra_pos in positivo:
    print("\n", palabra_pos.upper(),"\n")
    
    b_norte_expresion = b_norte_filtrado[(b_norte_filtrado['descripcion'].str.lower().str.contains(palabra_pos))]

    print("b_norte_pos filtrado cantidad: ", b_norte_expresion.shape[0])
    print()
    if len(b_norte_expresion)>0:
        print(b_norte_expresion.iloc[0]["descripcion"])
    else:
        print("No hubieron resultados")
    print("#################################","\n")


 LUMINOSO 

b_norte_pos filtrado cantidad:  562

Los equipos de aire acondicionado no estan incluidos en el precio de venta.Semipiso. Living. Comedor. 4 dormitorio(s) 1 en suite. 2 baño(s) 1 toilette(s). Entre medianeras. Al frente. S/av. principal.  Antiguo.  Luminoso.  En muy buen estado. Orientación Suroeste. Antigüedad 80 años. 
################################# 


 ESPACIOSO 

b_norte_pos filtrado cantidad:  10

Corredor Responsable: Uno Bienes Raices SRL - CUCICBA 4993 Contacto: Oscar Damián Esman - MLS ID # 420041128-232Departamento de Categoria a media Cuadra de Av. Santa Fe sobre calle Anchorena.  Semipiso 2AMB y 1/2 al frente con dependencia, balcón en un Segundo piso. Pleno Recoleta!  Espacioso Living-comedor con un pequeño ambiente que se puede usar como escritorio o playroom abierto. con frente a la calle Anchorena y contrafrente a pulmón.    Baño completo con disposición en suite o también, fuera de la habitación. Tiene un antebaño con doble circulación lo cual permite t

In [76]:

interior_inmueble = ["living", "comedor", "dormitorio", "baño", "cochera", "sum", "sala", "jardin", "jardín", "balcon", "balcón"]



In [77]:
b_norte_expresion = b_norte_filtrado.copy()
b_norte_anios = b_norte_expresion.copy()
for palabra_pos in interior_inmueble:
    print("\n", palabra_pos.upper(),"\n")
    
    b_norte_expresion = b_norte_filtrado[(b_norte_filtrado['descripcion'].str.lower().str.contains(palabra_pos))]

    print("b_norte_pos filtrado cantidad: ", b_norte_expresion.shape[0])
    print()
    if len(b_norte_expresion)>0:
        print(b_norte_expresion.iloc[0]["descripcion"])
    else:
        print("No hubieron resultados")
    print("#################################","\n")


 LIVING 

b_norte_pos filtrado cantidad:  946

Los equipos de aire acondicionado no estan incluidos en el precio de venta.Semipiso. Living. Comedor. 4 dormitorio(s) 1 en suite. 2 baño(s) 1 toilette(s). Entre medianeras. Al frente. S/av. principal.  Antiguo.  Luminoso.  En muy buen estado. Orientación Suroeste. Antigüedad 80 años. 
################################# 


 COMEDOR 

b_norte_pos filtrado cantidad:  939

Los equipos de aire acondicionado no estan incluidos en el precio de venta.Semipiso. Living. Comedor. 4 dormitorio(s) 1 en suite. 2 baño(s) 1 toilette(s). Entre medianeras. Al frente. S/av. principal.  Antiguo.  Luminoso.  En muy buen estado. Orientación Suroeste. Antigüedad 80 años. 
################################# 


 DORMITORIO 

b_norte_pos filtrado cantidad:  1008

Los equipos de aire acondicionado no estan incluidos en el precio de venta.Semipiso. Living. Comedor. 4 dormitorio(s) 1 en suite. 2 baño(s) 1 toilette(s). Entre medianeras. Al frente. S/av. principal.  Anti

In [96]:
caracteristicas_inmueble = ["antigüedad", "antiguedad", "semipiso", "pozo", "anio", " año", "a estrenar", "nuevo", " piso"]

In [97]:
b_norte_expresion = b_norte_filtrado.copy()
b_norte_anios = b_norte_expresion.copy()
for palabra_pos in caracteristicas_inmueble:
    print("\n", palabra_pos.upper(),"\n")
    
    b_norte_expresion = b_norte_filtrado[(b_norte_filtrado['descripcion'].str.lower().str.contains(palabra_pos))]

    print("b_norte_pos filtrado cantidad: ", b_norte_expresion.shape[0])
    print()
    if len(b_norte_expresion)>0:
        print(b_norte_expresion.iloc[0]["descripcion"])
    else:
        print("No hubieron resultados")
    print("#################################","\n")


 ANTIGÜEDAD 

b_norte_pos filtrado cantidad:  68

Los equipos de aire acondicionado no estan incluidos en el precio de venta.Semipiso. Living. Comedor. 4 dormitorio(s) 1 en suite. 2 baño(s) 1 toilette(s). Entre medianeras. Al frente. S/av. principal.  Antiguo.  Luminoso.  En muy buen estado. Orientación Suroeste. Antigüedad 80 años. 
################################# 


 ANTIGUEDAD 

b_norte_pos filtrado cantidad:  15

Muy lindo Monoambiente Moderno y luminoso sobre calle laprida entre juncal y french Recoleta .2do Piso Frente Superficie Cubierta 32mts mas 3,5 de Balcon corrido (entra una mesa pequeÃ±a y 2 sillas ) posee cocina completa estilo americana con desayunador, piso flotante , 1 baÃ±o completo y calefaccion del tipo split frio calor todo en impecable estado y super moderno el edificio cuenta ademas con Laundry , piscina , solarium , sum y parrilla.antiguedad aproximada 5 aÃ±os .expensas bajas 1500opcion venta con muebles.+ info  lineas rotativaso a 
##########################

# Palabras de amenities objetivos

In [ ]:
amenities = ["cocina integrada", "lavadero", "al frente", " frente", ]

In [118]:
palabra_clave = r"(\d+).{0,4}(piso)"
anio_expression = re.compile(palabra_clave)

b_norte_anios = b_norte_filtrado.copy()

anios_mask = (b_norte_anios['descripcion'].str.lower().str.contains(palabra_clave, regex=True))
print(len(anios_mask))

b_norte_anios["has_dummy"] = np.where(anios_mask, True, False)

b_norte_anios["dummy"] = np.where(anios_mask, b_norte_anios['descripcion'].str.lower().str.extract("(.{0,10}"+ palabra_clave + ".{0,10})"), np.nan)

b_norte_anios.head(40)

/Users/xavier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
/Users/xavier/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  # Remove the CWD from sys.path while we load stuff.


1660


ValueError: operands could not be broadcast together with shapes (1660,) (1660,3) () 

In [ ]:
alto_palermo_coords = (34.3518, 58.24378) #34°35'18.3"S 58°24'37.8"W
facultad_medicina_coords = (34.35507, 58.23539) #34°35'50.7"S 58°23'53.9"W

In [ ]:
def points_distance(point_1, point_2):
    x_1 = point_1[0]
    y_1 = point_1[1]
    x_2 = point_2[0]
    y_2 = point_2[1]
    distance = np.sqrt(((x_1-x_2)**2) + ((y_1-y_2)**2))
    return distance

# Limpiar el dataset
Practiquemos nuestras capacidades para realizar limpieza de datos con el dataset de ventas de propiedades de Properatti. Si no recuerdan cómo hacer alguna de estas tareas, busquen en Internet o en clases anteriores de manipulación de datos.

Buscamos que completen las siguientes tareas:
* Quitar columnas redundantes
* Convertir las variables categóricas numéricas a integer
* Lidiar con los valores perdidos y/o erróneos en algunas variables clave: 

    + en este punto podría ser interesante realizar algo ligeramente más complejo que un simple rellenado en función de medias. Podría eventualmente imputar en función de medias condicionadas.
        * por ejemplo, los campos `lat` y `long` tienen, también, muchos casos perdidos. Quizás una estrategia posible sería tratar de imputar las coordenadas con la media de los casos en el mismo `place_name` -barrio-.
    + también podrían evaluar la posibilidad de extraer alguna información para completar los datos perdidos del campo "description" que contiene texto del aviso. 
        * por ejemplo, el campo `rooms` tiene una altísima cantidad de missing. Podrían intentar usando alguna expresión regex extraer información 

In [ ]:
b_norte_df.shape

# Filtrar los Datos

En general, los mdoelos de precios suelen tener un carácter local. Es por eso que deberán elegir alguna zona o provincia para trabajar y estimar el modelo en base a dicha selección.

In [ ]:
# Filtrar la zona elegida para estimar el modelo



# Análisis exploratorio

Como estamos intentando predecir precios por metro cuadrado es importante realizar una primera vista y exploración de los datos.

In [ ]:
# Calculo de correlacion entre features


In [ ]:
# Analisis exploratorio, graficos...


¿En qué barrios los precios por $m^2$ son más altos? 

¿En qué barrios hay una mayor cantidad de viviendas para su venta?

¿ En qué barrios hay una mayor dispersión de precios?

¿Qué tipo de propiedad presenta los mayores precios? ¿Cuál las mayores dispersiones?

Continua realizando un análisis exploratorio del dataset...

## BONUS: Empezando con regresiones lineales...

Si se atreven, podrían empezar a probar con algunos modelos de regresión no demasiado sofisticados. Seleccionen (o construyan) su variable dependiente ($Y$) y comiencen probando con pocas variables y modelos sencillos.

Usen el estimador `LinearRegression` de submódulo `sklearn.linear_model`.

Recuerden los pasos para comenzar a trabajar con estimadores en Scikit-Learn.
    
1. Elegir una clase de modelo importando la clase de estimador apropiado de Scikit-Learn.
2. Seleccionar los hiperparámetros del modelo instanciando la clase con los valores deseados.
3. Preparar los datos en una matriz de features y un array target, como vimos previamente.
4. Ajustar el modelo a los datos invocando el método fit() de la instancia del modelo.
5. Aplicar el modelo a nuevos datos:
    * Para aprendizaje supervisado, frecuentemente predecimos labels para datos nuevos usando el método predict()
    * Para aprendizaje no supervisado, frecuentemente transformamos o inferimos propiedades de los datos usando los métodos transform() o predict()
    
¿Qué pueden decir del ajuste y la capacidad predictiva de sus modelos?

In [ ]:
from sklearn.linear_model import LinearRegression